In [18]:
# 1. Instalasi
!pip install -q streamlit imblearn

# ============================================
# Proyek Klasifikasi Obesitas - UAS
# Tahapan: EDA, Preprocessing, Modeling, Tuning, Deployment (Streamlit)
# ============================================

# 1. Instalasi (hapus komentar saat dijalankan di Colab)
# !pip install -q streamlit imblearn

# 2. Import Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import joblib
import io

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE

# 3. Load Data
df = pd.read_csv("ObesityDataSet.csv")

# 4. EDA (Exploratory Data Analysis)
st.title("Exploratory Data Analysis (EDA)")

st.write("**Data Sample:**")
st.write(df.head())

st.write("**Informasi Umum:**")
buffer = io.StringIO()
df.info(buf=buffer)
s = buffer.getvalue()
st.text(s)

st.write("**Missing Values:**")
st.write(df.isnull().sum())

st.write("**Jumlah Duplikat:**", df.duplicated().sum())

st.write("**Distribusi Kelas (Target):**")
st.bar_chart(df['NObeyesdad'].value_counts())

# 5. Preprocessing
st.title("Preprocessing")

# Konversi kolom numerik
num_cols = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop duplikasi dan missing values
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

# Label Encoding untuk kolom kategorikal
cat_cols = df.select_dtypes(include='object').columns
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])

# Split fitur dan label
X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

# Tangani imbalance dengan SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

# Normalisasi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_res, test_size=0.2, random_state=42)

# 6. Modeling & Evaluation
st.title("Modeling dan Evaluasi")

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier()
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Report": classification_report(y_test, y_pred, output_dict=True),
        "Confusion": confusion_matrix(y_test, y_pred)
    }

# Tampilkan hasil
for name, res in results.items():
    st.subheader(name)
    st.write("Accuracy:", res["Accuracy"])
    st.write("Classification Report:")
    st.dataframe(pd.DataFrame(res["Report"]).transpose())
    st.write("Confusion Matrix:")
    st.write(res["Confusion"])

# 7. Hyperparameter Tuning (contoh pada Random Forest)
st.title("Hyperparameter Tuning")

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=3, n_jobs=-1)
grid.fit(X_train, y_train)

best_rf = grid.best_estimator_
y_pred_best = best_rf.predict(X_test)

st.subheader("Random Forest (Optimized)")
st.write("Best Parameters:", grid.best_params_)
st.write("Accuracy:", accuracy_score(y_test, y_pred_best))
st.write("Classification Report:")
st.dataframe(pd.DataFrame(classification_report(y_test, y_pred_best, output_dict=True)).transpose())

# ============================================
# 8. Simpan Model
# ============================================
joblib.dump(best_rf, "rf_model.pkl")
joblib.dump(scaler, "scaler.pkl")

st.success("✅ Model dan Scaler berhasil disimpan!")



2025-06-12 09:30:43.037 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:30:43.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:30:43.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:30:43.041 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:30:43.047 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:30:43.049 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:30:43.050 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-12 09:30:43.051 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()